In [15]:
import matplotlib.pyplot as plt
from datetime import datetime
import folium
import numpy as np
import matplotlib as mpl
from folium.plugins import MarkerCluster
import json
import fiona 
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.ticker as ticker
import pandas as pd
from plotly import graph_objs as go
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import contextily as ctx

In [51]:
#import WAZE data
df=pd.read_csv('MORA_park_region_results_clean_20210817.csv', encoding="ISO-8859-1")
df.rename( columns={'Unnamed: 0':'Id'}, inplace=True )
#converting WAZE data frame into GeoDataFrame with geometry point 
alertPoints= df.loc[df['Type']!='in-park']
alertPoints = gpd.GeoDataFrame(alertPoints, geometry=gpd.points_from_xy(alertPoints.Lon, alertPoints.Lat),crs={'init' :'epsg:4326'})
alertPoints=alertPoints.to_crs(epsg=4326)


In [52]:
alertPoints.head()

,Id,NAME,UNIT_TYPE,Time,Alert,Sub_Alert,Street,State,Lon,Lat,Type,geometry
40719,40720,MORA,National Park,2017-03-05 12:53:57,ROAD_CLOSED,ROAD_CLOSED_EVENT,SE Green Valley Rd,WA,-122.034848,47.284619,region,POINT (-122.03485 47.28462)
40720,40721,MORA,National Park,2017-03-05 12:53:57,ROAD_CLOSED,ROAD_CLOSED_EVENT,SE Green Valley Rd,WA,-122.034848,47.284619,region,POINT (-122.03485 47.28462)
40721,40722,MORA,National Park,2017-03-05 12:54:07,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-410,WA,-121.420253,46.908198,region,POINT (-121.42025 46.90820)
40722,40723,MORA,National Park,2017-03-05 12:54:01,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-410,WA,-121.393264,46.914699,region,POINT (-121.39326 46.91470)
40723,40724,MORA,National Park,2017-03-05 12:54:05,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-410,WA,-121.539917,46.866887,region,POINT (-121.53992 46.86689)


In [53]:
alertPoints.count()

Id           268872
NAME         268872
UNIT_TYPE    268872
Time         268872
Alert        268872
Sub_Alert    258515
Street       266709
State        268872
Lon          268872
Lat          268872
Type         268872
geometry     268872
dtype: int64

In [54]:
#opening entrances shapefile
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
entrances = gpd.read_file('MORA Park Entrances.kml', driver='KML')

In [55]:
outline = gpd.read_file('NPS_-_Land_Resources_Division_Boundary_and_Tract_Data_Service.shp')
MORAoutline= outline.loc[outline['UNIT_CODE']=='MORA']
MORAoutline=MORAoutline.to_crs(epsg=4326)
MORAparkAlerts=gpd.clip(alertPoints,MORAoutline)
MORAparkAlerts.count()

Id           40719
NAME         40719
UNIT_TYPE    40719
Time         40719
Alert        40719
Sub_Alert    40703
Street       40667
State        40719
Lon          40719
Lat          40719
Type         40719
geometry     40719
dtype: int64

In [56]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
region = gpd.read_file('MORA Region.kml', driver='KML')
MORAAlerts=gpd.clip(alertPoints,region)

In [57]:
MORAAlerts.count()

Id           118475
NAME         118475
UNIT_TYPE    118475
Time         118475
Alert        118475
Sub_Alert    115979
Street       117890
State        118475
Lon          118475
Lat          118475
Type         118475
geometry     118475
dtype: int64

In [58]:
#clipping WAZE points to those within the entrance
MORAalerts=gpd.clip(alertPoints,entrances)
#Spatial join to tie entrance name to alert 
alerts_with_entrance=gpd.sjoin(MORAalerts,entrances,how="inner", op='intersects')
alerts_with_entrance.drop(columns=['index_right', 'Description'], inplace=True)
alerts_with_entrance.rename(columns={'Name': 'Entrance'}, inplace=True)

In [59]:
MORAparkAlerts['Park']=1
MORAparkAlerts['Region']=0
MORAAlerts['Park']=0
MORAAlerts['Region']=1

In [60]:
allAlerts=pd.concat([MORAparkAlerts, MORAAlerts], ignore_index=True)
alerts=allAlerts.drop_duplicates(subset=['Id'])

In [61]:
totalP = alerts['Park'].sum()
print(totalP)

40719


In [62]:
totalR = alerts['Region'].sum()
print(totalR)

77756


In [63]:
alerts.count()

Id           118475
NAME         118475
UNIT_TYPE    118475
Time         118475
Alert        118475
Sub_Alert    115979
Street       117890
State        118475
Lon          118475
Lat          118475
Type         118475
geometry     118475
Park         118475
Region       118475
dtype: int64

In [25]:
alerts_with_entrance.columns

Index(['Id', 'NAME', 'UNIT_TYPE', 'Time', 'Alert', 'Sub_Alert', 'Street',
       'State', 'Lon', 'Lat', 'Type', 'geometry', 'Entrance'],
      dtype='object')

In [64]:
enter=alerts_with_entrance.drop(columns=['NAME', 'UNIT_TYPE', 'Time', 'Alert', 'Sub_Alert', 'Street',
       'State', 'Lon', 'Lat', 'Type', 'geometry',])

In [65]:
enter

,Id,Entrance
40724,40725,SR-410 and Sunrise Rd
40745,40746,SR-410 and Sunrise Rd
41954,41955,SR-410 and Sunrise Rd
43459,43460,SR-410 and Sunrise Rd
43908,43909,SR-410 and Sunrise Rd
...,...,...
202331,202332,Mowich Lake
207010,207011,Mowich Lake
207012,207013,Mowich Lake
207104,207105,Mowich Lake


In [66]:
allAlerts=alerts.join(enter.set_index('Id'), on='Id')
allAlerts

,Id,NAME,UNIT_TYPE,Time,Alert,Sub_Alert,Street,State,Lon,Lat,Type,geometry,Park,Region,Entrance
0,40724,MORA,National Park,2017-03-05 12:54:05,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-410,WA,-121.539917,46.866887,region,POINT (-121.53992 46.86689),1,0,NaN
1,40732,MORA,National Park,2017-03-05 12:54:06,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-410,WA,-121.520425,46.869337,region,POINT (-121.52043 46.86934),1,0,Cayuse Pass
2,40734,MORA,National Park,2017-03-05 12:53:46,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-123,WA,-121.519142,46.830834,region,POINT (-121.51914 46.83083),1,0,NaN
3,40735,MORA,National Park,2017-03-05 12:54:33,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-123,WA,-121.554361,46.785824,region,POINT (-121.55436 46.78582),1,0,NaN
4,40737,MORA,National Park,2017-03-05 12:53:40,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-123,WA,-121.554361,46.785824,region,POINT (-121.55436 46.78582),1,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159189,309582,MORA,National Park,2021-08-14 20:58:31,ACCIDENT,NaN,SR-410,WA,-122.058314,47.160017,region,POINT (-122.05831 47.16002),0,1,NaN
159190,309583,MORA,National Park,2021-08-14 20:58:31,ACCIDENT,NaN,SR-410,WA,-122.058314,47.160017,region,POINT (-122.05831 47.16002),0,1,NaN
159191,309586,MORA,National Park,2021-08-15 22:34:45,WEATHERHAZARD,HAZARD_ON_SHOULDER_CAR_STOPPED,US-12,WA,-121.709705,46.566126,region,POINT (-121.70970 46.56613),0,1,NaN
159192,309587,MORA,National Park,2021-08-15 22:34:45,WEATHERHAZARD,HAZARD_ON_SHOULDER_CAR_STOPPED,US-12,WA,-121.709705,46.566126,region,POINT (-121.70970 46.56613),0,1,NaN


In [67]:
alerts_with_entrance.count()

Id           65910
NAME         65910
UNIT_TYPE    65910
Time         65910
Alert        65910
Sub_Alert    65811
Street       65884
State        65910
Lon          65910
Lat          65910
Type         65910
geometry     65910
Entrance     65910
dtype: int64

In [68]:
allAlerts.count()

Id           118475
NAME         118475
UNIT_TYPE    118475
Time         118475
Alert        118475
Sub_Alert    115979
Street       117890
State        118475
Lon          118475
Lat          118475
Type         118475
geometry     118475
Park         118475
Region       118475
Entrance      65910
dtype: int64

In [69]:
allAlerts.to_csv('MORAalertsTotal.csv')

In [70]:
allAlerts.head()

,Id,NAME,UNIT_TYPE,Time,Alert,Sub_Alert,Street,State,Lon,Lat,Type,geometry,Park,Region,Entrance
0,40724,MORA,National Park,2017-03-05 12:54:05,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-410,WA,-121.539917,46.866887,region,POINT (-121.53992 46.86689),1,0,NaN
1,40732,MORA,National Park,2017-03-05 12:54:06,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-410,WA,-121.520425,46.869337,region,POINT (-121.52043 46.86934),1,0,Cayuse Pass
2,40734,MORA,National Park,2017-03-05 12:53:46,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-123,WA,-121.519142,46.830834,region,POINT (-121.51914 46.83083),1,0,NaN
3,40735,MORA,National Park,2017-03-05 12:54:33,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-123,WA,-121.554361,46.785824,region,POINT (-121.55436 46.78582),1,0,NaN
4,40737,MORA,National Park,2017-03-05 12:53:40,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-123,WA,-121.554361,46.785824,region,POINT (-121.55436 46.78582),1,0,NaN
